# Classification of a twin boundary
---

This notebook demostrates classification of a 4DSTEM dataset containing a twin boundary.  The following steps are performed:
- Load data
- Initial visualization and virtual imaging
- Bragg disk detection
- Classification

### Data
The 4DSTEM data of a twin boundary used in this notebook was collected by Shiteng Zhao.

To download the data, please [go here](https://drive.google.com/file/d/1sUrPEgM1wWyTh-LJ30lGUhcXklHj6ajC/view?usp=sharing).  Assuming you're running the notebook on your local computer, you should then need to place the file somewhere on your filesystem, and in the cell immediately after this one, update the variable `filepath_input` to reflect that path to the file.


### Version info

Last updated on 2019-09-08 with py4DSTEM version 0.9.17.

In [ ]:
# Update this cell so that `fp_input` points to the data on your filesystem

filepath_input = "/Users/Ben/Work/Data/py4DSTEM_sampleData/classification_twinBoundary/twinBoundary_ShitengZhao20190115MEA.h5"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import py4DSTEM

## Load and examine the data

The next few cells load the datacube and vacuum probe, perform some basic analysis and visualization to  inspect each.

In [ ]:
# Examine what's inside the data file

py4DSTEM.file.io.read(filepath_input)

In [ ]:
# Load the data

datacube,_  = py4DSTEM.file.io.read(filepath_input,data_id='datacube')             # Get the 4D datacube

_probe,_ = py4DSTEM.file.io.read(filepath_input,data_id='probe')                   # Get the vacuum probe
probe = _probe.slices['probe']                                               # Unpack probe / probe kernel
probe_kernel = _probe.slices['probe_kernel']

#### Max diffraction pattern

A quick and useful way to examine a 4DSTEM datacube is to look at the maximal diffraction pattern - 
an array `dp_max` with the shape of the detector, in which the value assigned to each pixel in `dp_max` is
the highest intensity the corresponding detector pixel sees over the entire 4D scan.  Any Bragg scattering
across the region of the scan will then be included in a single diffraction image.

In [ ]:
dp_max = np.max(datacube.data,axis=(0,1))                                  # Get the maximal diffraction pattern
py4DSTEM.visualize.show(dp_max,1,4)                                        # Show

In [ ]:
# Generate a bright field image

qx0,qy0 = 208,212                                                                     # Define detector geometry
R = 18

im_bf = py4DSTEM.process.virtualimage.get_virtualimage_circ(datacube,qx0,qy0,R)       # Get image  

py4DSTEM.visualize.show_circ(dp_max,1,4,center=(qx0,qy0),R=R,color='r',alpha=0.5)     # Show detector
py4DSTEM.visualize.show(im_bf,5,5)                                                      # Show image

In [ ]:
# Inspect individual diffraction patterns

rx1,ry1 = 3,4                                                                # Select scan positions
rx2,ry2 = 6,21
dp1 = datacube.data[rx1,ry1,:,:]                                            # Get DPs
dp2 = datacube.data[rx2,ry2,:,:]
colors=['r','b']

fig,ax = py4DSTEM.visualize.show(im_bf,5,5,returnfig=True)                   # Show selected positions
ax.scatter((ry1,ry2),(rx1,rx2),color=colors,s=500)
plt.show()
py4DSTEM.visualize.show_image_grid(lambda i:[dp1,dp2][i],1,2,axsize=(5,5),         # Show DPs
                             get_bc=lambda i:colors[i])

#### Virtual dark-field imaging

Looking at the diffraction patterns on the left and right of the scan, we can identify a twin boundary.  In `dp_max`, its easy to see where the twinned scattering vectors are.  So with `dp_max` in hand, we can use the same code that generated a bright field image, above, to make virtual dark-field images which light up the two twins present in this data.  Arguably, this is enough to call this data successfully classified.

In [ ]:
# Virtual dark-field imaging

qx0_d1,qy0_d1 = 172,194                                                                  # Define detector geometries
qx0_d2,qy0_d2 = 174,180
R = 7

im_df1 = py4DSTEM.process.virtualimage.get_virtualimage_circ(datacube,qx0_d1,qy0_d1,R)  # Get virtual DF images
im_df2 = py4DSTEM.process.virtualimage.get_virtualimage_circ(datacube,qx0_d2,qy0_d2,R)

py4DSTEM.visualize.show_circ(dp_max,1,4,center=[(qx0_d1,qy0_d1),(qx0_d2,qy0_d2)],        # Show detectors
                                        R=R,color=colors,alpha=0.5,figsize=(8,8))
py4DSTEM.visualize.show_image_grid(lambda i:[im_df1,im_df2][i],2,1,axsize=(8,3),               # Show DF images
                             contrast='minmax',get_bc=lambda i:colors[i])

#### The vacuum probe

We also loaded `probe` and `probe_kernel`.
`probe` is the electron probe in diffraction space over vacuum.
`probe_kernel` is the vacuum probe with some additional processing to prepare it for the Bragg disk detection step.
Preparing vacuum probes and kernels are discussed elsewhere - see TKTKTK.

In [ ]:
py4DSTEM.visualize.show(probe,contrast='minmax',figsize=(6,6))                           # Show the probe
fig,ax = py4DSTEM.visualize.show(probe,contrast='minmax',figsize=(6,6),returnfig=True)   # Show it again,
ax.set_ylim(200,216)                                                                     # this time zooming in
ax.set_xlim(205,221)                                                                     # to the center
plt.show()

In [ ]:
py4DSTEM.visualize.show_kernel(probe_kernel,R=10,L=80,W=2)            # Show the probe kernel

## Bragg disk detection

In this step we find all the Bragg disks in the dataset.  The approach is to cross correlate the vacuum probe with each diffraction pattern, identifying the Bragg peak positions with the cross correlation maxima.

This step is computationally expensive, and for most datasets can be quite slow (this example dataset is very small!).  It's usually a good idea to optimize parameters on a few test diffraction patterns before running the computation on the entire datacube.

In [ ]:
# Select a few diffraction patterns to test parameters on

rx1_,ry1_ = 3,4                                                                # Select scan positions
rx2_,ry2_ = 6,21
dp1_ = datacube.data[rx1_,ry1_,:,:]                                             # Get DPs
dp2_ = datacube.data[rx2_,ry2_,:,:]
colors=['r','b']

fig,ax = py4DSTEM.visualize.show(im_bf,5,5,returnfig=True)                     # Show selected positions
ax.scatter((ry1_,ry2_),(rx1_,rx2_),color=colors,s=500)
plt.show()
py4DSTEM.visualize.show_image_grid(lambda i:[dp1_,dp2_][i],1,2,axsize=(5,5),           # Show DPs
                                   get_bc=lambda i:colors[i])

In [ ]:
# Tune parameters and detect disks on the selected diffraction patterns

corrPower = 1
sigma = 5
edgeBoundary = 20
maxNumPeaks = 50
minPeakSpacing = 10
minRelativeIntensity = 0.05
relativeToPeak = 1
subpixel = 'poly'

peaks = py4DSTEM.process.diskdetection.find_Bragg_disks_selected(             # Find Bragg peaks for a few
                      datacube,                                               # selected diffraction patterns
                      probe_kernel,
                      (rx1_,rx2_), (rx2_,ry2_),
                      corrPower=corrPower,
                      sigma=sigma,
                      edgeBoundary=edgeBoundary,
                      minRelativeIntensity=minRelativeIntensity,
                      relativeToPeak=relativeToPeak,
                      minPeakSpacing=minPeakSpacing,
                      maxNumPeaks=maxNumPeaks,
                      subpixel=subpixel)

py4DSTEM.visualize.show_points(dp1_,x=peaks[0].data['qx'],y=peaks[0].data['qy'],s=peaks[0].data['intensity'],
                               point_color='r',bordercolor='r',min=1,max=4,figsize=(8,8))
py4DSTEM.visualize.show_points(dp2_,x=peaks[1].data['qx'],y=peaks[1].data['qy'],s=peaks[1].data['intensity'],
                               point_color='b',bordercolor='b',min=1,max=4,figsize=(8,8))

In [ ]:
# Perform the full computation, using the parameters currently in memory

braggpeaks = py4DSTEM.process.diskdetection.find_Bragg_disks(
                              datacube,
                              probe_kernel,
                              corrPower=corrPower,
                              sigma=sigma,
                              edgeBoundary=edgeBoundary,
                              minRelativeIntensity=minRelativeIntensity,
                              relativeToPeak=relativeToPeak,
                              minPeakSpacing=minPeakSpacing,
                              maxNumPeaks=maxNumPeaks,
                              subpixel=subpixel,
                              verbose=True)
braggpeaks.name = 'braggpeaks'

#### Bragg vector map


Following disk detection, we can compute the Bragg vector map, defined as the sum of all the detected Bragg disk centers, weighted by their cross correlation intensities.
The BVM is roughly interpretable as a density map of Bragg scattering vectors.

In [ ]:
braggvectormap = py4DSTEM.process.diskdetection.get_bragg_vector_map(braggpeaks,datacube.Q_Nx,datacube.Q_Ny)

py4DSTEM.visualize.show(braggvectormap,0,5,cmap='jet')

## Calibration

Depending on the dataset, some calibration may be helpful at this point, including correction of diffraction shifts and/or elliptical distortions.  These are essential in many datasets.  Calibration of the detector pixel size and the rotational offset of real/reciprocal space can also be performed now.

In this dataset, calibration is not necessary to complete the classification, so will be skipped here for succinctness.  Sample calibration code and discussion can be found elsewhere - see TKTKTK.

## Classification

In this step we perform the classification using the detected Bragg peaks.  The algorithm used here is as follows:

1. Maxima in the Bragg vector map are used to segment k-space using a Voronoi tesselation.  Let's call the number of regions `N`.
- For each diffraction pattern, a feature vector is contructed consisting of `N` Boolean values, indicating whether Bragg scattering was detected through each k-space region in each diffraction pattern.
- An initial classification is performed by identifying scan positions which are most likely to have the same Bragg peaks
- The initial classification is refined using non-negative matrix factorization

With the classes in hand, we also compute the class diffraction patterns.

In [ ]:
Qx, Qy, intensity = py4DSTEM.process.utils.get_maxima_2D(                    # Find the maxima of the BVM
                                braggvectormap,
                                sigma=1,
                                edgeBoundary=20,
                                minSpacing=8,
                                minRelativeIntensity=0.005,
                                relativeToPeak=1,
                                maxNumPeaks=100,
                                subpixel=True)
N = len(Qx)                                                                  # The feature vector length

py4DSTEM.visualize.show_points(braggvectormap,x=Qx,y=Qy,s=intensity,min=0,max=5,cmap='jet')

In [ ]:
py4DSTEM.visualize.show_voronoi(braggvectormap,Qx,Qy,min=0,max=2,cmap='jet')         # Display the voronoi tesselation

In [ ]:
# Select a maximum distance from the BVM maxima that a given Bragg peak can be and still be considered
# an instance of this peak

max_dist = 12

py4DSTEM.visualize.show_voronoi(braggvectormap,Qx,Qy,min=0,max=2,cmap='jet',max_dist=max_dist)

In [ ]:
classification = py4DSTEM.process.classification.BraggVectorClassification(          # Set up classification Class
                        braggpeaks,Qx,Qy,max_dist=max_dist,X_is_boolean=True)

In [ ]:
classification.get_initial_classes_by_cooccurrence(                     # Generate initial classes.
                        thresh=0.3,
                        BP_fraction_thresh=0.1,
                        max_iterations=5)

In [ ]:
# Show initial class images

colors=['r','b']

py4DSTEM.visualize.show_image_grid(lambda i: classification.get_class_image(i),2,1,axsize=(8,3),
                                contrast='minmax',get_bc=lambda i: colors[i],titlesize=18)

In [ ]:
# Show class bragg peak weights

py4DSTEM.visualize.show_class_BPs_grid(braggvectormap,H=1,W=2,x=Qx,y=Qy,s2=25,
                                       min=0,max=2,cmap='jet',titlesize=18,
                                       get_s=lambda i:200*classification.get_class_BPs(i),
                                       get_bc=lambda i:colors[i])

In [ ]:
# Compute the class diffraction patterns

class_dps = np.zeros((datacube.Q_Nx,datacube.Q_Ny,classification.N_c))
for i in range(classification.N_c):
    class_dps[:,:,i] = py4DSTEM.process.classification.get_class_DP(
                                datacube,
                                classification.get_class_image(i)
    )
    
py4DSTEM.visualize.show_image_grid(lambda i: class_dps[:,:,i],
                                   1,2,axsize=(6,6),cmap='gray',min=1,max=4,get_bc=lambda i: colors[i])

In [ ]:
# Calculate a dark reference to perform background subtraction

darkref = py4DSTEM.process.preprocess.get_darkreference(datacube,N_frames=50,width_x=10,width_y=10)
py4DSTEM.visualize.show(darkref,contrast='minmax',figsize=(6,6))

In [ ]:
# Recompute class diffraction patterns

class_dps = np.zeros((datacube.Q_Nx,datacube.Q_Ny,classification.N_c))
for i in range(classification.N_c):
    class_dps[:,:,i] = py4DSTEM.process.classification.get_class_DP(
                                datacube,
                                classification.get_class_image(i),
                                darkref=darkref
    )
    
py4DSTEM.visualize.show_image_grid(lambda i: class_dps[:,:,i],
                                   1,2,axsize=(6,6),cmap='gray',min=1,max=4,get_bc=lambda i: colors[i])

In [ ]:
# Refine classes using non-negative matrix factorization

classification.nmf(max_iterations=200)
classification.accept()

In [ ]:
# Show the classes

py4DSTEM.visualize.show_image_grid(lambda i: classification.get_class_image(i),2,1,axsize=(8,3),
                                   contrast='minmax',get_bc=lambda i: colors[i],titlesize=18)

In [ ]:
# Show class bragg peak weights

py4DSTEM.visualize.show_class_BPs_grid(braggvectormap,H=1,W=2,x=Qx,y=Qy,s2=25,min=0,max=2,cmap='jet',titlesize=18,
                                       get_s=lambda i:200*classification.get_class_BPs(i),get_bc=lambda i:colors[i])

In [ ]:
# Compute class diffraction patterns

class_dps = np.zeros((datacube.Q_Nx,datacube.Q_Ny,classification.N_c))
for i in range(classification.N_c):
    class_dps[:,:,i] = py4DSTEM.process.classification.get_class_DP(
                                datacube,
                                classification.get_class_image(i),
                                darkref=darkref
    )
    
py4DSTEM.visualize.show_image_grid(lambda i: class_dps[:,:,i],
                                   1,2,axsize=(6,6),cmap='gray',min=1,max=4,get_bc=lambda i: colors[i])